In [5]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

# Load model architecture from JSON file
with open("fer2.json", "r") as json_file:
    loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)

# Load weights into the model
model.load_weights('model_filter.h5')

face_haar_cascade = cv2.CascadeClassifier('haarcascades_haarcascade_frontalface_default.xml')

capture = cv2.VideoCapture(0)

while True:
    # Capture frame and return boolean value and captured image
    ret, frame = capture.read()
    if not ret:
        continue
    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)

    for (x, y, w, h) in faces_detected:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), thickness=2)
        # Cropping region of interest, i.e., the face area from the image
        roi_gray = gray_img[y:y+w, x:x+h]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        # Find the emotion with maximum confidence
        max_index = np.argmax(predictions[0])

        emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
        predicted_emotion = emotions[max_index]

        cv2.putText(frame, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    resized_img = cv2.resize(frame, (1000, 700))
    cv2.imshow('Facial emotion analysis', resized_img)

    if cv2.waitKey(10) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step


...

1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 12ms/step


In [4]:
# Load model architecture from JSON file
with open("fer2.json", "r") as json_file:
    loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)

# Load weights into the model
model.load_weights('model_filter.h5')

# Access the layers and their weights
for layer in model.layers:
    layer_name = layer.name
    layer_weights = layer.get_weights()
    print("Layer Name:", layer_name)
    print("Layer Weights:", layer_weights)
    print()
    


Layer Name: conv2d_1
Layer Weights: [array([[[[-3.20813917e-02, -6.13628700e-02, -4.47459780e-02, ...,
          -1.23999827e-01,  8.74376204e-03, -2.31596977e-02]],

        [[-6.60249265e-03, -2.29445938e-02,  6.57728780e-03, ...,
           2.84605678e-02, -7.59174079e-02,  2.18630359e-02]],

        [[ 1.77761505e-03,  3.57305584e-03,  2.44929679e-02, ...,
           8.48411694e-02, -9.39693078e-02, -4.59212363e-02]],

        [[-2.88783815e-02,  2.65386049e-02,  9.73492116e-02, ...,
           5.58716729e-02, -2.29541995e-02,  1.68351065e-02]],

        [[-1.87816331e-04, -5.67504652e-02,  4.16765846e-02, ...,
          -8.23033005e-02,  6.46752417e-02, -3.48477997e-03]]],


       [[[ 1.96285807e-02, -1.15676679e-01, -2.72606444e-02, ...,
          -2.84730893e-04,  2.78315116e-02, -4.65673096e-02]],

        [[ 1.01546142e-02, -1.01559930e-01,  6.83603212e-02, ...,
           1.29587859e-01, -1.51151577e-02,  1.69438682e-02]],

        [[-5.91791794e-02, -3.14150862e-02, -9.5407

In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        1664      
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 64)        102464    
                                                                 
 batch_normalization_1 (Bat  (None, 48, 48, 64)        256       
 chNormalization)                                                
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 24, 24, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 128)       204928    
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 128)      

In [6]:
import h5py

def explore_hdf5(file, prefix='', max_depth=3, current_depth=0):
    for key in file.keys():
        path = f'{prefix}/{key}'
        if current_depth >= max_depth:
            print(f'Reached max exploration depth at: {path}')
            continue

        if isinstance(file[key], h5py.Group):
            print(f'Group: {path}')
            explore_hdf5(file[key], prefix=path, max_depth=max_depth, current_depth=current_depth + 1)
        elif isinstance(file[key], h5py.Dataset):
            dataset = file[key]
            print(f'Dataset: {path}')
            print(f'Shape: {dataset.shape}')
            # You can choose whether to print values or not
        else:
            print(f'Unknown type: {path}')

# Open the HDF5 file
file_path = 'model_filter.h5'
with h5py.File(file_path, 'r') as file:
    explore_hdf5(file)


Group: /model_weights
Group: /model_weights/activation_1
Group: /model_weights/activation_2
Group: /model_weights/batch_normalization_1
Group: /model_weights/batch_normalization_1/batch_normalization_1
Reached max exploration depth at: /model_weights/batch_normalization_1/batch_normalization_1/beta:0
Reached max exploration depth at: /model_weights/batch_normalization_1/batch_normalization_1/gamma:0
Reached max exploration depth at: /model_weights/batch_normalization_1/batch_normalization_1/moving_mean:0
Reached max exploration depth at: /model_weights/batch_normalization_1/batch_normalization_1/moving_variance:0
Group: /model_weights/batch_normalization_2
Group: /model_weights/batch_normalization_2/batch_normalization_2
Reached max exploration depth at: /model_weights/batch_normalization_2/batch_normalization_2/beta:0
Reached max exploration depth at: /model_weights/batch_normalization_2/batch_normalization_2/gamma:0
Reached max exploration depth at: /model_weights/batch_normalization